This notebook merges two separate data files toghether. This is useful for us in the data exploration and cleaning step of ML.

In [17]:
%run -i ../helpers.py
%matplotlib inline
import pandas as pd
pd.options.display.max_columns = 999

[nltk_data] Downloading package punkt to /Users/z.askary/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use mps:0
Device set to use mps:0


## Load transcript data

In [19]:
# Load the transcript data from a pickle file
tp = pd.read_pickle('../data/motley-fool-data.pkl')

# Parse the 'date' column to datetime and create a new 'dt' column
tp['dt'] = tp.date.apply(lambda x: parse_dt(x))

# Drop the original 'date' and 'exchange' columns as they are no longer needed
tp.drop(columns = ['date','exchange'],inplace=True)

# Load the S&P 500 companies data from a CSV file
snp = pd.read_csv('../data/sp500_companies.csv')

# Ensure the 'Symbol' column in the S&P 500 data is of type string
snp.Symbol = snp.Symbol.astype(str)

# Create a list of S&P 500 companies for which we have transcripts
constituents_snp = [x for x in list(tp.ticker.unique()) if x in snp.Symbol.tolist()]

# Print the number of S&P 500 companies for which we have transcripts
print(f'we have {len(constituents_snp)} companies from S&P 500 for which we have transcripts.')

we have 465 companies from S&P 500 for which we have transcripts.


In [20]:
print(tp.ticker.nunique()) # number of companies in kaggle dataset

2876


In [21]:
print(tp[tp.ticker=='GOOGL'].iloc[0].transcript)

Prepared Remarks:
Operator
Welcome, everyone. Thank you for standing by for the Alphabet Second Quarter 2021 Earnings Conference Call. [Operator Instructions]
I would now like to hand the conference over to your speaker today, Jim Friedland, Director of Investor Relations. Please go ahead.
Jim Friedland -- Director of Investor Relations
Thank you. Good afternoon everyone and welcome to Alphabet's second quarter earnings conference call. With us today are Sundar Pichai, Philipp Schindler and Ruth Porat.
Now, I'll quickly cover the Safe Harbor. Some of the statements that we make today regarding our business, operations and financial performance, including the effect of the COVID-19 pandemic on those areas may be considered forward-looking and such statements involve a number of risks and uncertainties that could cause actual results to differ materially. For more information, please refer to the risk factors discussed in our most recent Form 10-K filed with the SEC.
During this call, we

In [22]:
tp.q.unique() #2018-2020 
# This line of code retrieves the unique values from the 'q' column of the 'tp' dataframe.
# The 'q' represents yearly quarters (i.e. Jan, Mar, Apr - ...)

array(['2020-Q2', '2020-Q3', '2020-Q1', '2019-Q3', '2019-Q2', '2021-Q2',
       '2022-Q2', '2022-Q1', '2022-Q3', '2020-Q4', '2021-Q4', '2021-Q3',
       '2021-Q1', '2018-Q4', '2022-Q4', '2019-Q4', '2023-Q1', '2023-Q2',
       '2019-Q1', '2023-Q3', '2018-Q2', '2018-Q3', '2018-Q1', '2017-Q4',
       '2017-Q3'], dtype=object)

In [23]:
# This dataframe contains information about S&P 500 companies
snp

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,254.49,3846819807232,1.346610e+11,0.061,Cupertino,CA,United States,164000.0,"Apple Inc. designs, manufactures, and markets ...",0.069209
1,NMS,NVDA,NVIDIA Corporation,NVIDIA Corporation,Technology,Semiconductors,134.70,3298803056640,6.118400e+10,1.224,Santa Clara,CA,United States,29600.0,NVIDIA Corporation provides graphics and compu...,0.059350
2,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software - Infrastructure,436.60,3246068596736,1.365520e+11,0.160,Redmond,WA,United States,228000.0,Microsoft Corporation develops and supports so...,0.058401
3,NMS,AMZN,"Amazon.com, Inc.","Amazon.com, Inc.",Consumer Cyclical,Internet Retail,224.92,2365033807872,1.115830e+11,0.110,Seattle,WA,United States,1551000.0,"Amazon.com, Inc. engages in the retail sale of...",0.042550
4,NMS,GOOGL,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,191.41,2351625142272,1.234700e+11,0.151,Mountain View,CA,United States,181269.0,Alphabet Inc. offers various products and plat...,0.042309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,NMS,CZR,"Caesars Entertainment, Inc.","Caesars Entertainment, Inc.",Consumer Cyclical,Resorts & Casinos,32.82,6973593600,3.668000e+09,-0.040,Reno,NV,United States,51000.0,"Caesars Entertainment, Inc. operates as a gami...",0.000125
498,NYQ,BWA,BorgWarner Inc.,BorgWarner Inc.,Consumer Cyclical,Auto Parts,31.88,6972155904,1.882000e+09,-0.048,Auburn Hills,MI,United States,39900.0,"BorgWarner Inc., together with its subsidiarie...",0.000125
499,NMS,QRVO,"Qorvo, Inc.","Qorvo, Inc.",Technology,Semiconductors,70.85,6697217024,6.731300e+08,-0.052,Greensboro,NC,United States,8700.0,"Qorvo, Inc. engages in development and commerc...",0.000120
500,NYQ,FMC,FMC Corporation,FMC Corporation,Basic Materials,Agricultural Inputs,50.15,6260525568,7.033000e+08,0.085,Philadelphia,PA,United States,5800.0,"FMC Corporation, an agricultural sciences comp...",0.000113


In [24]:
# tp_snp = tp[tp.ticker.isin(constituents_snp)]

# Print the shape of the original dataframe 'tp'
print(tp.shape)

# Merge the 'tp' dataframe with the 'snp' dataframe on the 'ticker' and 'Symbol' columns, keeping only the rows that have matching values in both dataframes
tp_snp = tp.merge(snp, left_on='ticker', right_on='Symbol', how='inner')

# Print the shape of the merged dataframe 'tp_snp'
print(tp_snp.shape)

# Print the number of unique tickers in the merged dataframe 'tp_snp'
print(tp_snp.ticker.nunique())

# Drop unnecessary columns from the merged dataframe 'tp_snp'
tp_snp.drop(columns = ['Exchange', 'Longname', 'City', 'State', 'Country'], inplace=True)

(18755, 4)
(3663, 20)
465


In [25]:
tp[tp.ticker=='AAPL'].q.unique()

array(['2020-Q2', '2022-Q3', '2020-Q1', '2020-Q3', '2019-Q3', '2021-Q1',
       '2021-Q2', '2020-Q4', '2021-Q3', '2022-Q1', '2021-Q4', '2023-Q1',
       '2022-Q2', '2022-Q4'], dtype=object)

In [26]:
print(tp.ticker.nunique())
tp.dt.describe() #between 2019-2023

2876


count                            18375
mean     2021-06-09 21:16:36.062040832
min                2019-04-11 10:00:00
25%                2021-02-02 08:00:00
50%                2021-07-27 11:00:00
75%                2021-12-02 09:45:00
max                2023-02-23 11:00:00
Name: dt, dtype: object

In [28]:
tp_snp.to_csv('../data/transcripts_sp_500_companies.csv',index=None)
# Save the merged dataframe 'tp_snp' to a CSV file named 'transcripts_sp_500_companies.csv' without including the index

In [29]:
def count_splits_transcripts(tr):
    """
    Counts the number of splits in the transcript based on the presence of specific keywords.

    Parameters:
    tr (str): The transcript text.

    Returns:
    int: 1 if 'questions and answers' or 'questions & answers' is found,
        2 if 'analyst' or 'portfolio manager' is found,
        0 otherwise.
    """
    qna = [x for x in sent_tokenize(tr) if 'questions and answers' in str.lower(x) or 'questions & answers' in str.lower(x)] 
    
    if(len(qna)>0):
        return 1
    else:
        qnaa = [x for x in sent_tokenize(tr) if 'analyst' in str.lower(x) or 'portfolio manager' in str.lower(x)] 
        if(len(qnaa)>0):
            return 2
        else:
            return 0
        

##### SPLITTING THE DATA
        
def split_transcript(row):
    """
    Splits the transcript into prepared remarks and Q&A sections based on specific keywords.

    Parameters:
    row (pd.Series): A row of the dataframe containing the transcript.

    Returns:
    pd.Series: The row with added 'prep_remarks' and 'QnA' fields.
    """
    splits = re.split(r'questions and answers|questions & answers', str.lower(row.transcript))
    if(len(splits)>0):
        row['prep_remarks'] = splits[0]
        row['QnA'] = ' '.join(splits[1:None])
    else:
        splits = re.split(r'analyst|portfolio manager', str.lower(row.transcript))
        if(len(splits)>0):
            row['prep_remarks'] = splits[0]
            row['QnA'] = ' '.join(splits[1:None])
        else:
            print('exception')
            row['prep_remarks'] = str.lower(row.transcript)
            row['QnA'] = str.lower(row.transcript)
    return(row)

In [30]:
# len(re.split('|questions \& answers|analyst|portfolio manager]', ))
# str.lower(tp_snp.iloc[0].transcript).find('questions and answers')

In [31]:
# Apply the split_transcript function to each row of the tp_snp dataframe
tp_splits = tp_snp.apply(lambda x: split_transcript(x), axis=1)

# Drop the 'transcript' column from the tp_splits dataframe as it is no longer needed
tp_splits.drop(columns='transcript', inplace=True)

# Print the shape of the dataframe where the 'QnA' column has missing values
print(tp_splits[tp_splits.QnA.isna()].shape)

# Save the resulting dataframe to a CSV file
tp_splits.to_csv('../data/tp_splits.csv', index=None)

(0, 16)


In [32]:
tp_splits[tp_splits.ticker=='ADI'][['ticker', 'q','dt' ,'prep_remarks','QnA',]].sort_values('dt', ascending=False).head()

,ticker,q,dt,prep_remarks,QnA
2924,ADI,2021-Q4,2021-11-23 10:00:00,"prepared remarks:\noperator\ngood morning, and...",:\noperator\n[operator instructions] your firs...
2751,ADI,2021-Q3,2021-08-18 10:00:00,"prepared remarks:\noperator\ngood morning, and...",:\noperator\n[operator instructions] your firs...
984,ADI,2021-Q2,2021-05-19 10:00:00,"prepared remarks:\noperator\ngood morning, and...",:\noperator\nthank you. [operator instructions...
1079,ADI,2021-Q1,2021-02-17 10:00:00,"prepared remarks:\noperator\ngood morning, and...",:\noperator\n[operator instructions] our first...
